# Advanced Q&A with AWS Bedrock with Langchain 

Welcome to this Jupyter notebook guide, where we delve into: Langchain, Chroma DB, and AWS Bedrock. This notebook is designed to walk you through the setup and application of these tools in a question-answering context, leveraging the strengths of each to create a robust and intelligent system.

- **Configuring AWS Bedrock:** AWS Bedrock's large language models will serve as the backbone of our system, providing the computational power and linguistic understanding necessary for processing complex queries.

- **Initializing Chroma DB Client:** Next, we'll establish a connection with Chroma DB, a vector database designed for efficient storage and retrieval of data. This step is crucial for managing the knowledge our model will access.

- **Setting Up Langchain:** We'll start by setting up Langchain, an innovative toolkit that allows us to seamlessly blend the capabilities of large language models with external databases.

- **Running a Practical Example:** Once our setup is complete, we'll run a demonstration function. This function showcases how to utilize Langchain, Bedrock, and Chroma DB in unison to answer questions. We'll use vector data stored in Chroma DB to feed our language model with relevant information, enhancing the accuracy and relevance of its responses.



## Configuring AWS Bedrock

Let's start by configuring our AWS credentials as a profile for Langchain.

The function below will use pre-set environment variables to create a credentials file with a **default** profile for Langchain.

In [1]:
import os

def create_aws_credentials_file():
    # Retrieve environment variables
    aws_region = os.environ.get('AWS_DEFAULT_REGION', 'default_region')
    aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID', 'default_access_key')
    aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY', 'default_secret_key')

    # Define the folder and file paths
    aws_folder_path = os.path.expanduser('~/.aws')
    credentials_file_path = os.path.join(aws_folder_path, 'credentials')

    # Create the .aws directory if it does not exist
    if not os.path.exists(aws_folder_path):
        os.makedirs(aws_folder_path)

    # Write the credentials to the file
    with open(credentials_file_path, 'w') as credentials_file:
        credentials_file.write('[default]\n')
        credentials_file.write(f'aws_access_key_id={aws_access_key_id}\n')
        credentials_file.write(f'aws_secret_access_key={aws_secret_access_key}\n')
        credentials_file.write(f'region={aws_region}\n')

    print(f"AWS credentials file created at: {credentials_file_path}")
    
create_aws_credentials_file()    

AWS credentials file created at: /home/cdsw/.aws/credentials


## Initializing Chroma

Next, we will configure the ChromaDB and use the same PersistentClient used in section 3c, please use the **3c_load_to_chroma_vector_db/populate_chroma_vectors.py** file to populate the vector DB with your data. 

We will use **cml-default** as the collection

In [2]:
import chromadb

persistent_client = chromadb.PersistentClient(path="/home/cdsw/chroma-data")
COLLECTION_NAME = "cml-default"

collection = persistent_client.get_or_create_collection(COLLECTION_NAME)

We will use [sentence-transformers/all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) model to embed our data in ChromaDB

In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"
EMBEDDING_MODEL_NAME = "all-mpnet-base-v2"

EMBEDDING_FUNCTION = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL_NAME)

## Getting started with Langchain 

Let's setup Langchain, we will start with configuring the Vector Store as Chroma using the persistent client we setup previously along with the embedding function and collection.

In [5]:
from langchain.vectorstores import Chroma

# Set up the Chroma vector store with the persistent client and collection name
vectorstore = Chroma(
        client=persistent_client,
        collection_name=COLLECTION_NAME,
        embedding_function=EMBEDDING_FUNCTION
    )

AWS Bedrock can be configured with Langchain using an AWS credentials profile and the model name to be used. We will be using the **anthropic.claude-v2:1** model for our example

In [7]:
from langchain.llms import Bedrock

LLM_MODEL = Bedrock(
    credentials_profile_name="default", model_id="anthropic.claude-v2:1"
)

Let's add a simple prompt which instructs our LLM to use the retrieved context when answering the question. We will pass the Prompt Template to our Question and Answering Chain

In [8]:
from langchain.prompts import PromptTemplate

# Prompt Template for Langchain
template = """You are a helpful AI assistant. Use only the below provided Context to answer the following question. If you do not know the answer respond with "I don't know."
Context:{context}
>>QUESTION<<{question}
>>ANSWER<<"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

Finally, we will pull everything together and create our Retrieval QA Chain using the Bedrock Model, Chroma Vectorstore and Prompt Template

In [9]:
from langchain.chains import RetrievalQA

# Create the QA chain with the Chroma vector store
qa_chain = RetrievalQA.from_chain_type(
        llm=LLM_MODEL,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    )

We can now call the QA Chain and ask questions to our documents. 

**Please set the QUESTION variable with your question.**

In [10]:

# Function to call the QA chain on the provided question and return the answer as a result 
def generate_response(question,qa_chain):
    result = qa_chain({"query": question})
    return result["result"]

# Run the QA chain and Access the result
QUESTION = "What does the Privacy Rule permit?"

print(f"Question:{QUESTION} \n \n Answer:{generate_response(QUESTION, qa_chain)}") 

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Question:What does the Privacy Rule permit? 
 
 Answer:The Privacy Rule permits, but does not require, a covered entity voluntarily to obtain patient consent for uses and disclosures of protected health information for treatment, payment, and health care operations.


## Complete Code

In [ ]:
import langchain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.llms import Bedrock
import chromadb


# Set up the persistent client
persistent_client = chromadb.PersistentClient(path="/home/cdsw/chroma-data")
COLLECTION_NAME = "cml-default"
# Create a collection in the persistent client
collection = persistent_client.get_or_create_collection(COLLECTION_NAME)

EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"
EMBEDDING_MODEL_NAME = "all-mpnet-base-v2"
EMBEDDING_FUNCTION = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL_NAME)

# Set up the Chroma vector store with the persistent client and collection name
vectorstore = Chroma(
        client=persistent_client,
        collection_name=COLLECTION_NAME,
        embedding_function=EMBEDDING_FUNCTION
    )


LLM_MODEL = Bedrock(
    credentials_profile_name="default", model_id="anthropic.claude-v2:1"
)

# Prompt Template for Langchain
template = """You are a helpful AI assistant. Use only the below provided Context to answer the following question. If you do not know the answer respond with "I don't know."
Context:{context}
>>QUESTION<<{question}
>>ANSWER<<"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


# Create the QA chain with the Chroma vector store
qa_chain = RetrievalQA.from_chain_type(
        llm=LLM_MODEL,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    )

def generate_response(question,qa_chain):
    result = qa_chain({"query": question})
    return result["result"]

# Run the QA chain and Access the result
QUESTION = "What does the Privacy Rule permit?"

print(f"Question:{QUESTION} \n \n Answer:{generate_response(QUESTION, qa_chain)}") 

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Question:What does the Privacy Rule permit? 
 
 Answer:The Privacy Rule permits, but does not require, a covered entity voluntarily to obtain patient consent for uses and disclosures of protected health information for treatment, payment, and health care operations.


## Conclusion

As we wrap up this Jupyter notebook, we've not only navigated through the process of setting up Langchain, Chroma DB, and AWS Bedrock but also successfully executed a question-answering function that leverages these technologies in unison. This has given us valuable insights into the integration of advanced language models with efficient data retrieval systems, showcasing the immense potential of AI in transforming how we interact with and process information.